In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

Using device: cuda


In [2]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:03<00:00, 47.2MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 4 * 4, 64)
        self.fc2 = nn.Linear(64, 10)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.maxpool(x)
        x = self.relu(self.conv2(x))
        x = self.maxpool(x)
        x = self.relu(self.conv3(x))
        x = self.maxpool(x)
        x = x.view(-1, 128 * 4 * 4)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

start_time = time.time()

num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}")

end_time = time.time()

elapsed_time = end_time - start_time
print(f"Training completed in {elapsed_time:.2f} seconds")

Epoch [1/25], Loss: 1.4568
Epoch [2/25], Loss: 1.0537
Epoch [3/25], Loss: 0.8839
Epoch [4/25], Loss: 0.7708
Epoch [5/25], Loss: 0.6878
Epoch [6/25], Loss: 0.6207
Epoch [7/25], Loss: 0.5569
Epoch [8/25], Loss: 0.5062
Epoch [9/25], Loss: 0.4579
Epoch [10/25], Loss: 0.4128
Epoch [11/25], Loss: 0.3706
Epoch [12/25], Loss: 0.3388
Epoch [13/25], Loss: 0.3009
Epoch [14/25], Loss: 0.2712
Epoch [15/25], Loss: 0.2412
Epoch [16/25], Loss: 0.2253
Epoch [17/25], Loss: 0.1996
Epoch [18/25], Loss: 0.1829
Epoch [19/25], Loss: 0.1622
Epoch [20/25], Loss: 0.1572
Epoch [21/25], Loss: 0.1452
Epoch [22/25], Loss: 0.1364
Epoch [23/25], Loss: 0.1235
Epoch [24/25], Loss: 0.1253
Epoch [25/25], Loss: 0.1084
Training completed in 358.85 seconds


In [12]:
start_time = time.time()
model.eval()
correct = 0
total = 0
y_true = []
y_pred = []
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())
print(f"Accuracy: {100 * correct / total:.2f}%")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Evaluation completed in {elapsed_time:.2f} seconds")

Accuracy: 73.21%
Evaluation completed in 2.17 seconds


In [11]:
print("Classification Report:\n", classification_report(y_true, y_pred, target_names=['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']))

Classification Report:
               precision    recall  f1-score   support

    Airplane       0.79      0.73      0.76      1000
  Automobile       0.81      0.87      0.84      1000
        Bird       0.63      0.65      0.64      1000
         Cat       0.53      0.59      0.56      1000
        Deer       0.69      0.69      0.69      1000
         Dog       0.60      0.63      0.62      1000
        Frog       0.80      0.78      0.79      1000
       Horse       0.83      0.75      0.79      1000
        Ship       0.83      0.86      0.84      1000
       Truck       0.87      0.77      0.82      1000

    accuracy                           0.73     10000
   macro avg       0.74      0.73      0.73     10000
weighted avg       0.74      0.73      0.73     10000

